### import Dataframe manipulating library

In [107]:
import pandas as pd

### Create a dataframe of movie watched by user

In [108]:
userinput = [
                {'title': 'A','rating':2},
                {'title': 'B','rating':10},
                {'title': 'C','rating':8}
                
]
user_movies_rating = pd.DataFrame(userinput)
user_movies_rating

,rating,title
0,2,A
1,10,B
2,8,C


In [109]:
user_movies_data  = [
                    {'title':'A', 'genres':'adventure|superhero'},
                    {'title':'B', 'genres':'comedy|adventure|superhero|scifi'},
                    {'title':'C', 'genres':'comedy|superhero'}
]
user_movies_data = pd.DataFrame(user_movies_data)
user_movies_data

,genres,title
0,adventure|superhero,A
1,comedy|adventure|superhero|scifi,B
2,comedy|superhero,C


### split the genre in form of list for making analysis easier

In [110]:
user_movies_data['genres'] = user_movies_data.genres.str.split('|')
user_movies_data

,genres,title
0,"[adventure, superhero]",A
1,"[comedy, adventure, superhero, scifi]",B
2,"[comedy, superhero]",C


### mark the genre 1 if it is present in the movie else mark 0

In [111]:
user_movies_with_genres = user_movies_data.copy()
for index, row in user_movies_data.iterrows():
    for genre in row['genres']:
        user_movies_with_genres.at[index, genre] = 1
        
user_movies_with_genres= user_movies_with_genres.fillna(0)
user_movies_with_genres

,genres,title,adventure,superhero,comedy,scifi
0,"[adventure, superhero]",A,1.0,1.0,0.0,0.0
1,"[comedy, adventure, superhero, scifi]",B,1.0,1.0,1.0,1.0
2,"[comedy, superhero]",C,0.0,1.0,1.0,0.0


### create the User profile based on which type of genre he watched

In [112]:
user_movies_with_genres = user_movies_with_genres.drop('genres', 1).drop('title', 1)
user_profile = user_movies_with_genres.transpose().dot(user_movies_rating['rating'])
user_profile

adventure    12.0
superhero    20.0
comedy       18.0
scifi        10.0
dtype: float64

### Create a dataframe of movies not watched by user

In [113]:
#movies not watched by user
movies = [
             {'title':'D', 'genres':'comedy|adventure|scifi','movieId': 1},
             {'title':'E', 'genres':'superhero','movieId': 2}
]
movies = pd.DataFrame(movies)
movies

,genres,movieId,title
0,comedy|adventure|scifi,1,D
1,superhero,2,E


In [114]:
movies['genres'] = movies.genres.str.split('|')
movies

,genres,movieId,title
0,"[comedy, adventure, scifi]",1,D
1,[superhero],2,E


In [115]:
movies_table = movies.copy()
for index, row in movies.iterrows():
    for genre in row['genres']:
        movies_table.at[index, genre] = 1

movies_table = movies_table.fillna(0)
movies_table

,genres,movieId,title,comedy,adventure,scifi,superhero
0,"[comedy, adventure, scifi]",1,D,1.0,1.0,1.0,0.0
1,[superhero],2,E,0.0,0.0,0.0,1.0


### Create genre table for not watched movies

In [117]:
genre_table = movies_table.set_index(movies_table['movieId'])
genre_table = genre_table.drop('genres', 1).drop('title', 1).drop('movieId', 1)
genre_table

,comedy,adventure,scifi,superhero
movieId,,,,
1,1.0,1.0,1.0,0.0
2,0.0,0.0,0.0,1.0


### get the recommendation table 

In [122]:
recommendation_table = ((genre_table*user_profile).sum(axis=1))/user_profile.sum()
recommendation_table = recommendation_table.sort_values(ascending =False)
recommendation_table

movieId
1    0.666667
2    0.333333
dtype: float64

In [124]:
print('recommended movie is:')
movies.loc[movies['movieId'].isin(recommendation_table.head(1).keys())]



recommended movie is:


,genres,movieId,title
0,"[comedy, adventure, scifi]",1,D
